## <a href="https://cursos.alura.com.br/course/langchain-python-ferramentas-llm-openai/task/156163"><b>Implementando uma SimpleSequentialChain para a criação de um roteiro de viagens</b><br/>(Técnica de Engenharia de Prompts Successive Prompts)</a>
##### <li>O objetivo será criar uma cadeia de chamadas, cada uma delas representando um prompt</li> 
##### <li>Uma interação de múltiplos prompts é um chat, por isso, vamos importar um ChatPromptTemplate</li>

In [ ]:
from langchain.prompts import ChatPromptTemplate # Uma interação de múltiplos prompts é um chat, por isso, vamos importar um ChatPromptTemplate

In [3]:
%pip install -qr requirements.txt

Note: you may need to restart the kernel to use updated packages.


#### <b>PASSO 1 - IMPORTS e CRIAÇÃO DA LLM</b>

In [7]:
from langchain_openai import ChatOpenAI
from os import getenv
from dotenv import load_dotenv # CARREGA A VARIÁVEL DE AMBIENTE OPENAI_KEY LIDA DO ARQUIVO .env

load_dotenv() # CARREGANDO O ARQUIVO COM A OPENAI_KEY

llm = ChatOpenAI( # INSTANCIANDO A LLM
                    model="gpt-4.1-mini",
                    temperature=0.5,
                    # 1 - OBTENDO A API KEY POR MEIO DA VARIÁVEL DE AMBIENTE OPENAI_KEY. QUE VAI FICAR ARMAZENADA NO ARQUIVO .env.
                    # 2 - AINDA É NECESSÁRIO CARREGAR ESSE ARQUIVO. VER NA PRIMEIRA CÉLULA DO NOTEBOOK
                    api_key=getenv("OPENAI_KEY")
                )

#### <b>PASSO 2 - CRIANDO O <i>CHAT PROMPT TEMPLATE</i></b></br>

<ul><li>Também possui várias formas de inicializar (Métodos from_template, from_messages (Já que é um chat, é o uso da técnica de Prompt Few-Shots)). <br/>
Assim como o PromptTemplate, <a href="https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html">ver na documentação as diferentes formas</a><br/>
<br>Vamos inicializar usando template (from_template)</br>

<b><ol><li>CRIANDO OS MODELOS</li></ol></b>

<ul><ul><li>CRIANDO O MODELO PARA A CIDADE</li></ul></ul>

In [17]:
template_cidade = ChatPromptTemplate.from_template("Sugira uma cidade, dado o meu interesse por {interesse}") # INSTANCIANDO ChatPromptTemplate e INICIANDO A PARTIR DE UM TEMPLATE

<ul><ul><li>CRIANDO O MODELO PARA O RESTAURANTE</li></ul></ul>

In [18]:
template_restaurante = ChatPromptTemplate.from_template("Sugira restaurantes populares entre locais na {cidade}") # INSTANCIANDO ChatPromptTemplate e INICIANDO A PARTIR DE UM TEMPLATE

<ul><ul><li>CRIANDO O MODELO CULTURAL</li></ul></ul>

In [19]:
template_cultural = ChatPromptTemplate.from_template("Sugira atividades e locais culturais em {cidade}") # INSTANCIANDO ChatPromptTemplate e INICIANDO A PARTIR DE UM TEMPLATE

<b><ol start="2"><li>CRIANDO AS CADEIAS (LLMChain) (Prompts)</li></ol></b> 

In [20]:
from langchain.chains import LLMChain

cadeia_cidade = LLMChain(llm=llm,prompt=template_cidade)
cadeia_restaurante = LLMChain(llm=llm,prompt=template_restaurante)
cadeia_cultural = LLMChain(llm=llm,prompt=template_cultural)

<b><ol start="3"><li>CRIANDO A CADEIA GERAL (SimpleSequentialChain)</li></ol></b>

In [32]:
from langchain.chains import SimpleSequentialChain

cadeia = SimpleSequentialChain(chains=[cadeia_cidade,cadeia_restaurante,cadeia_cultural])

#### <b>PASSO 4 - INVOCANDO A CADEIA GERAL</b>

In [ ]:
resposta = cadeia.invoke(input="praias") # DIFERENTE DO QUE ACONTECEU COM O PROMPT TEMPLATE, AQUI INVOCAMOS A CADEIA, E COMTÉM UM TEMPLATE, E ELA INVOCA A LLM
print(resposta)

{'input': 'praias', 'output': 'Claro! Além das praias e dos ótimos restaurantes que você mencionou, Florianópolis também oferece uma rica cena cultural e diversas atividades para quem quer conhecer mais sobre a história, arte e tradições locais. Aqui vão algumas sugestões culturais e de lazer:\n\n### Atividades culturais em Florianópolis:\n\n1. **Centro Histórico de Florianópolis**  \n   Explore o centro da cidade, onde você pode visitar a Catedral Metropolitana, o Mercado Público (ótimo para experimentar comidas típicas e comprar artesanato) e o Palácio Cruz e Sousa, que abriga o Museu Histórico de Santa Catarina.\n\n2. **Museu Victor Meirelles**  \n   Localizado no centro histórico, esse museu é dedicado ao pintor catarinense Victor Meirelles e apresenta obras de arte e exposições temporárias.\n\n3. **Projeto Tamar – Centro de Visitantes Florianópolis**  \n   Para quem gosta de natureza e conservação marinha, o Projeto Tamar oferece uma oportunidade de conhecer mais sobre as tartarug

#### <b>OBTENDO APENAS O CONTEÚDO DA MENSAGEM</b>

In [37]:
print(resposta['output'])

Claro! Além das praias e dos ótimos restaurantes que você mencionou, Florianópolis também oferece uma rica cena cultural e diversas atividades para quem quer conhecer mais sobre a história, arte e tradições locais. Aqui vão algumas sugestões culturais e de lazer:

### Atividades culturais em Florianópolis:

1. **Centro Histórico de Florianópolis**  
   Explore o centro da cidade, onde você pode visitar a Catedral Metropolitana, o Mercado Público (ótimo para experimentar comidas típicas e comprar artesanato) e o Palácio Cruz e Sousa, que abriga o Museu Histórico de Santa Catarina.

2. **Museu Victor Meirelles**  
   Localizado no centro histórico, esse museu é dedicado ao pintor catarinense Victor Meirelles e apresenta obras de arte e exposições temporárias.

3. **Projeto Tamar – Centro de Visitantes Florianópolis**  
   Para quem gosta de natureza e conservação marinha, o Projeto Tamar oferece uma oportunidade de conhecer mais sobre as tartarugas marinhas e os esforços para protegê-las

##### <b>COMO FOI PASSADA A CIDADE PARA OS PROMPTS DE RESTAURANTE E CULTURAL ?</b>

<ul><li><b>USANDO O MODO VERBOSE DA SimpleSequentialChain</b></li></ul>

In [38]:
cadeia = SimpleSequentialChain(chains=[cadeia_cidade,cadeia_restaurante,cadeia_cultural],verbose=True)
resposta = cadeia.invoke(input="praias") 
print(resposta)



> Entering new SimpleSequentialChain chain...
Claro! Se você gosta de praias, uma ótima sugestão é Florianópolis, em Santa Catarina, Brasil. A cidade é conhecida por suas praias paradisíacas, como a Praia Mole, Joaquina e Campeche, além de oferecer uma boa infraestrutura turística, opções de esportes aquáticos e uma vida noturna animada. Você gostaria de uma sugestão em outro país ou região?
Claro! Se você está em Florianópolis e quer experimentar restaurantes populares entre os locais, aqui vão algumas sugestões que combinam com o clima praiano e a cultura da região:

1. **Box 32** – Localizado no Mercado Público de Florianópolis, é famoso pelos frutos do mar frescos e pratos típicos da culinária catarinense, como a sequência de camarão.

2. **Ostradamus** – Na Lagoa da Conceição, é conhecido pelas ostras frescas e pratos elaborados com frutos do mar, além de um ambiente agradável e vista para a lagoa.

3. **Bar do Arante** – Na praia do Pântano do Sul, é um clássico para quem quer 

##### <b>AINDA NÃO ESTÁ EXPLICADO COMO FOI PASSADA A VARIÁVEL CIDADE ENTRE OS PROMPTS</b>

<ul><li><b>USANDO O MODO DEBUG DA SimpleSequentialChain</b></li></ul>

In [40]:
from langchain.globals import set_debug
set_debug(True)

resposta = cadeia.invoke(input="praias")
print(resposta)

[chain/start] [chain:SimpleSequentialChain] Entering Chain run with input:
{
  "input": "praias"
}
[chain/start] [chain:SimpleSequentialChain > chain:LLMChain] Entering Chain run with input:
{
  "interesse": "praias"
}
[llm/start] [chain:SimpleSequentialChain > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Sugira uma cidade, dado o meu interesse por praias"
  ]
}
[llm/end] [chain:SimpleSequentialChain > chain:LLMChain > llm:ChatOpenAI] [3.09s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Claro! Se você gosta de praias, uma ótima sugestão é Florianóp

<ul><li><b>CONCLUSÃO</b></li></ul>
<ul><b>Resposta de cidade</b></ul>

<ul>
[<font style="color: orange">llm/start</font>] [chain:SimpleSequentialChain > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:</br>
{</br>
  <ul>"prompts": [</ul>
    <ul><ul>"Human: Sugira uma <b>cidade</b>, dado o meu interesse por praias" </ul></ul>
  <ul>]</ul>
}</br></br>

[<font style="color: cyan">chain/end</font>] [chain:SimpleSequentialChain > chain:LLMChain] [3.09s] Exiting Chain run with <b>output</b>:</br>
{</br>
  <ul>"text": <b>"Claro! Se você gosta de praias, uma ótima sugestão é Florianópolis, em Santa Catarina, Brasil. A cidade é famosa por suas praias lindas, que atendem a diversos gostos,</br>
  desde praias com ondas para surfistas, como a Praia Mole e a Joaquina, até praias mais calmas e familiares, como a Praia da Daniela e a Lagoinha do Leste. Além disso, Florianópolis tem
  uma ótima infraestrutura, boa gastronomia e muitas opções de lazer. Quer que eu sugira alguma outra cidade, talvez em outro país?"</b></ul>
}
</ul>

<ul><li><b>LIGOU a variável "cidade" A RESPOSTA DADA.</b> (Aproveitou a resposta anterior para a entrada para a cadeia seguinte (Cadeia restaurante que se inicia))</li></ul></ul>

<ul>
[<font style="color: cyan">chain/start</font>] [chain:SimpleSequentialChain > chain:LLMChain] Entering Chain run with input:</br>
{</br>
  <ul><b>"cidade"</b>: "Claro! Se você gosta de praias, uma ótima sugestão é Florianópolis, em Santa Catarina, Brasil. A cidade é famosa por suas praias lindas, que atendem a diversos gostos, desde </br>
  praias com ondas para surfistas, como a Praia Mole e a Joaquina, até praias mais calmas e familiares, como a Praia da Daniela e a Lagoinha do Leste. Além disso,
  Florianópolis tem uma ótima infraestrutura, boa gastronomia e muitas opções de lazer. Quer que eu sugira alguma outra cidade, talvez em outro país?"</ul>
}</ul>


<ul><b>Substituindo a varíavel "cidade" no prompt pela resposta</b></ul>

<ul>
  [<font style="color: orange">llm/start</font>] [chain:SimpleSequentialChain > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:</br>
  {</br>
    <ul>"prompts": [</br>
      <ul>"Human: Sugira restaurantes populares entre locais na <b>Claro! Se você gosta de praias, uma ótima sugestão é Florianópolis, em Santa Catarina, Brasil. A cidade é famosa por suas praias lindas, que atendem a diversos gostos, desde praias com ondas para surfistas, como a Praia Mole e a Joaquina, até praias mais calmas e familiares, como a Praia da Daniela e a Lagoinha do Leste. Além disso, Florianópolis tem uma ótima infraestrutura, boa gastronomia e muitas opções de lazer. Quer que eu sugira alguma outra cidade, talvez em outro país?"</b></ul>
    </ul><ul>]</ul>
  }</br>

  [<font style="color: orange">llm/end</font>] [chain:SimpleSequentialChain > chain:LLMChain > llm:ChatOpenAI] [4.79s] Exiting LLM run with output:</ul></br>

<ul><li><b>PARA A CADEIA CULTURAL, A VARÍAVEL CIDADE, SERÁ SUBSTITUÍDA PELA RESPOSTA DO PROMPT cadeia_restaurante</b></li></ul>

<ul>[<font style="color: cyan">chain/start</font>] [chain:SimpleSequentialChain > chain:LLMChain] Entering Chain run with input:</br>
{<br>
  <ul><b>"cidade"</b>: "Claro! Se você está em Florianópolis e quer experimentar restaurantes populares entre os locais, aqui vão algumas sugestões que combinam ótima comida e ambiente autêntico:\n\n1. **Ostradamus** – Localizado na Lagoa da Conceição, é famoso pelos pratos com frutos do mar frescos, especialmente ostras e mariscos, além de uma bela vista da lagoa.\n\n2. **Box 32** – Também na Lagoa da Conceição, é um lugar pequeno e simples, mas com comida típica catarinense deliciosa, especialmente frutos do mar.\n\n3. **Bar do Boni** – Tradicional bar no centro de Florianópolis, ótimo para petiscos e um chopp gelado, muito frequentado por moradores.\n\n4. **Restaurante Rancho Açoriano** – Oferece pratos típicos da culinária açoriana, que é a base da cultura local, com destaque para o arroz de marisco e o peixe fresco.\n\n5. **Ponto Gula** – Um bistrô moderno com pratos autorais e ingredientes locais, perfeito para quem quer algo diferente e bem elaborado.\n\nSe quiser, posso sugerir cidades e restaurantes em outros destinos pelo Brasil ou pelo mundo. É só avisar!"</ul>
}</br></br>
[<font style="color: orange">llm/start</font>] [chain:SimpleSequentialChain > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input: </br>
{
  <ul>"prompts": [</br>
    <ul>"Human: Sugira atividades e locais culturais em <b>Claro! Se você está em Florianópolis e quer experimentar restaurantes populares entre os locais, aqui vão algumas sugestões que combinam ótima comida e ambiente autêntico:\n\n1. **Ostradamus** – Localizado na Lagoa da Conceição, é famoso pelos pratos com frutos do mar frescos, especialmente ostras e mariscos, além de uma bela vista da lagoa.\n\n2. **Box 32** – Também na Lagoa da Conceição, é um lugar pequeno e simples, mas com comida típica catarinense deliciosa, especialmente frutos do mar.\n\n3. **Bar do Boni** – Tradicional bar no centro de Florianópolis, ótimo para petiscos e um chopp gelado, muito frequentado por moradores.\n\n4. **Restaurante Rancho Açoriano** – Oferece pratos típicos da culinária açoriana, que é a base da cultura local, com destaque para o arroz de marisco e o peixe fresco.\n\n5. **Ponto Gula** – Um bistrô moderno com pratos autorais e ingredientes locais, perfeito para quem quer algo diferente e bem elaborado.\n\nSe quiser, posso sugerir cidades e restaurantes em outros destinos pelo Brasil ou pelo mundo. É só avisar!"</b></ul>
  ]</ul>
}</br>
[<font style="color: orange">llm/end</font>] [chain:SimpleSequentialChain > chain:LLMChain > llm:ChatOpenAI] [9.10s] Exiting LLM run with output:

##### <b>UMA MANEIRA DE MELHORAR <i>SOMENTE</i> A SAÍDA DO PRIMEIRO PROMPT, PARA SER APROVEITADO NO SEGUNDO, SERIA:</b>

In [41]:
template_cidade = ChatPromptTemplate.from_template("Sugira uma cidade, dado o meu interesse por {interesse}. Informe SOMENTE a cidade") # INSTANCIANDO ChatPromptTemplate e INICIANDO A PARTIR DE UM TEMPLATE

cadeia_cidade = LLMChain(llm=llm,prompt=template_cidade)
cadeia_restaurante = LLMChain(llm=llm,prompt=template_restaurante)
cadeia_cultural = LLMChain(llm=llm,prompt=template_cultural)

cadeia = SimpleSequentialChain(chains=[cadeia_cidade,cadeia_restaurante,cadeia_cultural],verbose=True)
resposta = cadeia.invoke(input="praias") 
print(resposta)

[chain/start] [chain:SimpleSequentialChain] Entering Chain run with input:
{
  "input": "praias"
}
[chain/start] [chain:SimpleSequentialChain > chain:LLMChain] Entering Chain run with input:
{
  "interesse": "praias"
}
[llm/start] [chain:SimpleSequentialChain > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Sugira uma cidade, dado o meu interesse por praias. Informe SOMENTE a cidade"
  ]
}
[llm/end] [chain:SimpleSequentialChain > chain:LLMChain > llm:ChatOpenAI] [1.29s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Florianópolis",
            "additio